# Custom Choropleth Maps with Plotly/Mapbox


The next project is to create a choropleth map using the polygons of the supply areas.

Like always we start by retrieving the information we need. In this case, we can get it directly from our REST API in the geojson format.

In [1]:
import requests

supply_r = requests.get('https://peridash.ml/api/supply-areas/')

Let's test a sample choropleth map to get used to the process.

In [7]:
import plotly.plotly as py
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

df['text'] = df['state'] + '<br>' +\
    'Beef '+df['beef']+' Dairy '+df['dairy']+'<br>'+\
    'Fruits '+df['total fruits']+' Veggies ' + df['total veggies']+'<br>'+\
    'Wheat '+df['wheat']+' Corn '+df['corn']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['code'],
        z = df['total exports'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Millions USD")
        ) ]

layout = dict(
        title = '2011 US Agriculture Exports by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )

Notice that we have to use the online library of plotly to create these maps. I still haven't figured out how to create maps in an offline manner. It probably is not possible, as we require information from external servers like the polygons of the US or the map tiles from mapbox, and this information is constantly being updated, so is not part of the plotly library.  

In [9]:
py.iplot(fig, filename='test-map')

Now that's a good looking map! However, to achieve this we rely on predefined maps (in this case of the US). For our map, we would like to define the boundary polygons ourselves, using our own geojson polygons. We also would like map tiles like the ones provided by mapbox.

But first things first. Let's convert our raw file to a python json object for easy handling.

In [10]:
import json

supply_json = json.loads(supply_r.text)

Now, if we want to control the coloring for each supply area individually, we need to split the geojson.

In [22]:
epsa_jsons = []

for feature in supply_json['features']:
    epsa_json = dict(
        type='FeatureCollection',
        features=[feature]
    )
    epsa_jsons.append(epsa_json)

With the individual jsons in place, we can now create a layer for each. These layers will then be placed over the mapbox tiles to complete the map. 

For the moment, the colouring will just cycle through a list of predefined colors, but if the map works out properly, it would be nice to color the supply-areas based on the values of some variable/indicator of the corresponding EPSA. To do this, we need to access the variables/indicators tables using the EPSA code as key.   

In [31]:
# Example of accessing the EPSA code through the corresponding geojson.

epsa_jsons[0]['features'][0]['properties']['code']

'CAPAG'

We need to define a color array specifying the color of each supply area.

In [48]:
# colors = ['some_initial_value' for k in epsa_jsons]

base_colors = [
    'rgb(171, 99, 250)',
    'rgb(230, 99, 250)',
    'rgb(99, 110, 250)',
    'rgb(25, 211, 243)',
]

# colors = [ej['features'][0]['properties']['code'] for ej in epsa_jsons]
colors = [base_colors[k%4] for k in range(len(epsa_jsons))]

# for k, epsa_json in enumerate(epsa_jsons):
# #     colors[k] = epsa_jsons[k]['features'][0]['properties']['code']
#     colors[k] = base_colors[k%4]

Now we can create our layers. Each layer is a dictionary of type 'geojson'.

In [46]:
layers=[dict(
    sourcetype = 'geojson',
    source = epsa_jsons[k],
    below = "water",
    type = 'fill',   
    color = colors[k],
    opacity = 0.8
) for k in range(len(epsa_jsons))]

Good. With our layers properly defined we can start departing from basic python objects and start thinking in terms of plotly and mapbox.

Not quite yet! Unfortunately plotly does not offer hover functionality for polygons. But we do want the map to display some information when we are hovering near a supply area. How do we do this?
Here's one solution: Plotly does offer hover functionality for points. So, if we can place invisible points inside our polygons, we can add hover functionality for those and it will seem as if the hover functionality would be there for the polygons.

If our polygons were convex, we could just take median values and ensure the point will be inside the polygons (and even in the center). But in our case the polygons are not convex, so we need to work around this difficulty. This is the idea:

1. Take a polygon and place a bounding rectangle around it. 
2. Generate random points inside the recatangle.
3. Filter those points, that lie inside the polygon.

To do this, we used the geometry library 'Shapely' and generated a bunch of points for each polygon. Let's load those points! 

In [51]:
with open('points.json', encoding='utf8') as f:
    points_json = json.load(f)

With the points generated, we can create lists of longitudes, latitudes and texts to specify our scattermap.

In [59]:
p_lats = []
p_lons = []
p_texts = []

for code in list(points_json.keys()):
    epsa_p = points_json[code]
    n = len(epsa_p['lats'])
    p_lats += epsa_p['lats']
    p_lons += epsa_p['lons']
    p_texts += [epsa_p['text'] for k in range(n)]

Great! Now we can fill a scattermapbox layer with these coordinates and specify the corresponding text. Note that we set the opacity of the markers to zero, that way they won't show in the map.

In [69]:
len(points_json['CAPAG']['lons'])

30

In [70]:
p_colors = []
for color in colors:
    p_colors += [color for k in range(30)]
    

scatter_dict = dict(
    type = 'scattermapbox',
    mode = 'markers',
    
    lat = p_lats, 
    lon = p_lons,
    text = p_texts,
    
    marker = dict(
        size = 50,
        opacity = 0,
        color= p_colors
    ),
    showlegend = False,
    hoverinfo = 'text',
)

Ok, now we should have everything we need. All that's left to do is specify our figure, sign in to plotly and request our map!

In [71]:
mapbox_access_token = 'pk.eyJ1Ijoic2VyZ2lvLWNodW1hY2Vyby1maSIsImEiOiJjamswOTUzeHkwMDk0M3dvNnJoeTByZGlpIn0.3mmjpLwDrIUcdJTowlCd1A'

layout = dict(
    title = 'Áreas de Cobertura de las EPSA Reguladas',
    font = dict(family='Balto'),
    autosize = False,
    width = 1200,
    height = 700,
    hovermode = 'closest',

    mapbox = dict(
        accesstoken = mapbox_access_token,
        layers = layers,
        bearing = 0,
        center = dict(
            lat = -17.833763, 
            lon = -65.540431,
        ),
        pitch = 0,
        zoom = 5,
        style = 'light'
    ) 
)

fig = dict(data=[scatter_dict], layout=layout)

Congratulations! Now try adjusting the following improvements:

- make the map bigger (done)
- make the color of the hover text match the color of the polygon (not working for some reason, I'll probably have to make the color definition dynamic anyways if we want to make a choropleth map.)
- extract the code out of the text in the point information and use it to collect specific variable/indicator information to color the polygons accordingly

In [74]:
import plotly.plotly as py
py.iplot(fig, filename='mapa-coberturas')

c:\users\sergio\projects\presentations\muestrario\venv\lib\site-packages\plotly\tools.py:1424: UserWarning:

Looks like you used a newline character: '\n'.

Plotly uses a subset of HTML escape characters
to do things like newline (<br>), bold (<b></b>),
italics (<i></i>), etc. Your newline characters 
have been converted to '<br>' so they will show 
up right on your Plotly figure!



Ok, now let's try to color the polygons according to some indicator/variable. For this lets pull the data from our REST API and generate a dataframe.

In [80]:
import requests
import pandas as pd

measurements_r = requests.get('https://peridash.ml/api/measurements/')
epsas_r = requests.get('https://peridash.ml/api/epsas/')

measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)

df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

In [87]:
val = df[(df.code == 'CAPAG') & (df.year == 2016)].ind8.iloc[0]
float(val)

81.71